In [1]:
import numpy as np

index = np.load("/home/jovyan/Sample_Based_Extension/WUSTL/best_defense_indices.npy", allow_pickle=True)
index

array([['baseline', 0, 'Def1-11 Combined', 0.9999916231419081,
        0.9951091814583067, array([2, 2, 2, ..., 2, 2, 2])],
       ['BIM', 0.01, 'Def1-11 Combined', 0.9999916231419081,
        0.9951091814583067, array([2, 2, 2, ..., 2, 2, 2])],
       ['BIM', 0.1, 'Def1-11 Combined', 0.9998827239867143,
        0.9600649800293866, array([1, 1, 1, ..., 1, 1, 1])],
       ['BIM', 0.2, 'Def1-11 Combined', 0.9996984331086939,
        0.9305545272093338, array([3, 3, 3, ..., 3, 3, 3])],
       ['BIM', 0.3, 'Def1-11 Combined', 0.9984796002563319,
        0.7518383136072314, array([5, 5, 5, ..., 5, 5, 5])],
       ['FGSM', 0.01, 'Def1-11 Combined', 0.9999916231419081,
        0.9951091814583067, array([2, 2, 2, ..., 2, 2, 2])],
       ['FGSM', 0.1, 'Def1-11 Combined', 0.9998701586995765,
        0.939002641264322, array([1, 1, 1, ..., 1, 1, 1])],
       ['FGSM', 0.2, 'Def1-11 Combined', 0.9971518682487759,
        0.8894642081911899, array([3, 3, 3, ..., 3, 3, 3])],
       ['FGSM', 0.3, 'Def

In [2]:
attack_models = [
    ("BIM", [0.1]),
    ("FGSM", [0.1]),
    ("PGD", [0.1]),
    ("DF", [0.1]),
    ("AutoPGD", [0.1]),
    ("ZOO", [0.1]),
    ("CaFA", [0.1]),
    ("SINIFGSM", [0.1]),
    ("VNIFGSM", [0.1]),
]

all_adv_examples = []

for attack_name, epsilons in attack_models:
    for epsilon in epsilons:
        filename = f'/home/jovyan/Sample_Based_Extension/WUSTL/transfer_attack/x_test_adv_{attack_name}_eps_{epsilon}.npy'
        x_test_adv = np.load(filename)
        all_adv_examples.append(x_test_adv)

x_test_adv_0_1 = np.concatenate(all_adv_examples, axis=0)


In [3]:
y_labels = []

for row in index:
    attack_name, eps = row[0], row[1]
    if eps == 0.1:
        y_array = row[-1]
        y_labels.append(y_array)

y_label_0_1 = np.concatenate(y_labels, axis=0)

unique_labels, counts = np.unique(y_label_0_1, return_counts=True)
print(unique_labels, counts)
valid_labels = unique_labels[counts > 15]
mask = np.isin(y_label_0_1, valid_labels)
x_test_adv_0_1 = x_test_adv_0_1[mask]
y_label_0_1 = y_label_0_1[mask]
print(np.unique(y_label_0_1, return_counts = True))

[ 1  2  3  4  5  6  7  8  9 10 11] [956483     15 481243  10597   4847 468586      4 223743    652   2579
     28]
(array([ 1,  3,  4,  5,  6,  8,  9, 10, 11]), array([956483, 481243,  10597,   4847, 468586, 223743,    652,   2579,
           28]))


In [4]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
label_encoder.fit(y_label_0_1)

y_label_0_1 = label_encoder.transform(y_label_0_1)

for index, class_label in enumerate(label_encoder.classes_):
    print(f"defense '{class_label}' encode {index}")

defense '1' encode 0
defense '3' encode 1
defense '4' encode 2
defense '5' encode 3
defense '6' encode 4
defense '8' encode 5
defense '9' encode 6
defense '10' encode 7
defense '11' encode 8


In [5]:
# Collect 1% of eps = 0.1 data
from sklearn.model_selection import train_test_split

_, X_1, _ , y_1 = train_test_split(x_test_adv_0_1, y_label_0_1, test_size=0.01, stratify=y_label_0_1, random_state=42)

In [6]:
X_1.shape

(21488, 42)

In [7]:
from xgboost import XGBClassifier

print('xgb')
xgb1 = XGBClassifier()
xgb1.fit(X_1, y_1)

print("RF")
from sklearn.ensemble import RandomForestClassifier
rf1 = RandomForestClassifier()
rf1.fit(X_1, y_1)

# print("DT")
# from sklearn.tree import DecisionTreeClassifier
# dt1 = DecisionTreeClassifier()
# dt1.fit(X_1, y_1)

xgb
RF


RandomForestClassifier()

In [8]:
import time
import numpy as np

y_test = np.load('/home/jovyan/Sample_Based_Extension/WUSTL/y_test.npy')

attack_models = [
    ("baseline", [0]),
    ("BIM", [0.01, 0.2, 0.3]),
    ("FGSM", [0.01, 0.2, 0.3]),
    ("PGD", [0.01, 0.2, 0.3]),
    ("DF", [0.01, 0.2, 0.3]),
    ("AutoPGD", [0.01, 0.2, 0.3]),
    ("ZOO", [0.01, 0.2, 0.3]),
    ("CaFA", [0.01, 0.2, 0.3]),
    ("SINIFGSM", [0.01, 0.2, 0.3]),
    ("VNIFGSM", [0.01, 0.2, 0.3]),
]

base_path_template = "/home/jovyan/Sample_Based_Extension/WUSTL/transfer_attack/x_test_adv_{attack}_eps_{eps}.npy"

from sklearn.model_selection import train_test_split

for attack_name, epsilons in attack_models:
    start_time = time.time()
    all_adv_examples = []
    all_y = []
    
    if attack_name != "baseline":
        for epsilon in epsilons:
            filename = f'/home/jovyan/Sample_Based_Extension/WUSTL/transfer_attack/x_test_adv_{attack_name}_eps_{epsilon}.npy'
            x_test_adv = np.load(filename)
            all_adv_examples.append(x_test_adv)
            all_y.append(y_test)
    
        X = np.concatenate(all_adv_examples, axis=0)
        y = np.concatenate(all_y, axis=0)
    else:
        X = np.load('/home/jovyan/Sample_Based_Extension/WUSTL/x_test.npy')
        y = np.load('/home/jovyan/Sample_Based_Extension/WUSTL/y_test.npy')

    y_pred = xgb1.predict(X)
    print(y_pred.shape)
    y_pred = label_encoder.inverse_transform(y_pred)
    unique_classes, count = np.unique(y_pred, return_counts=True)
    print(unique_classes, count)
    for class_num in unique_classes:
        indices = np.where(y_pred == class_num)[0]
        X_class = X[indices]
        y_class = y[indices]
        print(X_class.shape, y_class.shape)
    
        x_filename = f"/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/XGB/WUSTL_Input1/x_test_adv_{attack_name}_Def{class_num}.npy"
        y_filename = f"/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/XGB/WUSTL_Input1/y_test_adv_{attack_name}_Def{class_num}.npy"
        
        np.save(x_filename, X_class)
        np.save(y_filename, y_class)
    
        print(f"Save {class_num} to {attack_name}_Def{class_num}.npy")

    end_time = time.time()
    result = end_time - start_time
    print(f"Execution Time: {result:.6f} seconds")
    

(238753,)
[3 6 8] [     7 237193   1553]
(7, 42) (7,)
Save 3 to baseline_Def3.npy
(237193, 42) (237193,)
Save 6 to baseline_Def6.npy
(1553, 42) (1553,)
Save 8 to baseline_Def8.npy
Execution Time: 0.345474 seconds
(716259,)
[ 1  3  5  6  8  9 10] [ 64451 252596   1439 170181 224811    398   2383]
(64451, 42) (64451,)
Save 1 to BIM_Def1.npy
(252596, 42) (252596,)
Save 3 to BIM_Def3.npy
(1439, 42) (1439,)
Save 5 to BIM_Def5.npy
(170181, 42) (170181,)
Save 6 to BIM_Def6.npy
(224811, 42) (224811,)
Save 8 to BIM_Def8.npy
(398, 42) (398,)
Save 9 to BIM_Def9.npy
(2383, 42) (2383,)
Save 10 to BIM_Def10.npy
Execution Time: 0.777718 seconds
(716259,)
[ 1  3  5  6  8  9 10] [444825  13431  11776  12876 225535      2   7814]
(444825, 42) (444825,)
Save 1 to FGSM_Def1.npy
(13431, 42) (13431,)
Save 3 to FGSM_Def3.npy
(11776, 42) (11776,)
Save 5 to FGSM_Def5.npy
(12876, 42) (12876,)
Save 6 to FGSM_Def6.npy
(225535, 42) (225535,)
Save 8 to FGSM_Def8.npy
(2, 42) (2,)
Save 9 to FGSM_Def9.npy
(7814, 42) (

In [9]:
from sklearn.model_selection import train_test_split

for attack_name, epsilons in attack_models:
    start_time = time.time()
    all_adv_examples = []
    all_y = []
    
    if attack_name != "baseline":
        for epsilon in epsilons:
            filename = f'/home/jovyan/Sample_Based_Extension/WUSTL/transfer_attack/x_test_adv_{attack_name}_eps_{epsilon}.npy'
            x_test_adv = np.load(filename)
            all_adv_examples.append(x_test_adv)
            all_y.append(y_test)
    
        X = np.concatenate(all_adv_examples, axis=0)
        y = np.concatenate(all_y, axis=0)
    else:
        X = np.load('/home/jovyan/Sample_Based_Extension/WUSTL/x_test.npy')
        y = np.load('/home/jovyan/Sample_Based_Extension/WUSTL/y_test.npy')

    y_pred = rf1.predict(X)
    print(y_pred.shape)
    y_pred = label_encoder.inverse_transform(y_pred)
    unique_classes, count = np.unique(y_pred, return_counts=True)
    print(unique_classes, count)
    for class_num in unique_classes:
        indices = np.where(y_pred == class_num)[0]
        X_class = X[indices]
        y_class = y[indices]
        print(X_class.shape, y_class.shape)
    
        x_filename = f"/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/RF/WUSTL_Input1/x_test_adv_{attack_name}_Def{class_num}.npy"
        y_filename = f"/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/RF/WUSTL_Input1/y_test_adv_{attack_name}_Def{class_num}.npy"
        
        np.save(x_filename, X_class)
        np.save(y_filename, y_class)
    
        print(f"Save {class_num} to {attack_name}_Def{class_num}.npy")

    end_time = time.time()
    result = end_time - start_time
    print(f"Execution Time: {result:.6f} seconds")
    

(238753,)
[3 6 8] [     7 237914    832]
(7, 42) (7,)
Save 3 to baseline_Def3.npy
(237914, 42) (237914,)
Save 6 to baseline_Def6.npy
(832, 42) (832,)
Save 8 to baseline_Def8.npy
Execution Time: 2.136139 seconds
(716259,)
[1 3 4 5 6 8] [227056 470890      1   1009    836  16467]
(227056, 42) (227056,)
Save 1 to BIM_Def1.npy
(470890, 42) (470890,)
Save 3 to BIM_Def3.npy
(1, 42) (1,)
Save 4 to BIM_Def4.npy
(1009, 42) (1009,)
Save 5 to BIM_Def5.npy
(836, 42) (836,)
Save 6 to BIM_Def6.npy
(16467, 42) (16467,)
Save 8 to BIM_Def8.npy
Execution Time: 2.741938 seconds
(716259,)
[ 1  3  5  6  8 10] [454108 222812   7860  11636  16749   3094]
(454108, 42) (454108,)
Save 1 to FGSM_Def1.npy
(222812, 42) (222812,)
Save 3 to FGSM_Def3.npy
(7860, 42) (7860,)
Save 5 to FGSM_Def5.npy
(11636, 42) (11636,)
Save 6 to FGSM_Def6.npy
(16749, 42) (16749,)
Save 8 to FGSM_Def8.npy
(3094, 42) (3094,)
Save 10 to FGSM_Def10.npy
Execution Time: 2.871321 seconds
(716259,)
[1 3 4 5 6 8] [227056 470890      1   1009   

In [10]:
# from sklearn.model_selection import train_test_split

# for attack_name, epsilons in attack_models:
#     start_time = time.time()
#     all_adv_examples = []
#     all_y = []
    
#     if attack_name != "baseline":
#         for epsilon in epsilons:
#             filename = f'/home/jovyan/Sample_Based_Extension/WUSTL/transfer_attack/x_test_adv_{attack_name}_eps_{epsilon}.npy'
#             x_test_adv = np.load(filename)
#             all_adv_examples.append(x_test_adv)
#             all_y.append(y_test)
    
#         X = np.concatenate(all_adv_examples, axis=0)
#         y = np.concatenate(all_y, axis=0)
#     else:
#         X = np.load('/home/jovyan/Sample_Based_Extension/WUSTL/x_test.npy')
#         y = np.load('/home/jovyan/Sample_Based_Extension/WUSTL/y_test.npy')

#     y_pred = dt1.predict(X)
#     print(y_pred.shape)
#     y_pred = label_encoder.inverse_transform(y_pred)
#     unique_classes, count = np.unique(y_pred, return_counts=True)
#     print(unique_classes, count)
#     for class_num in unique_classes:
#         indices = np.where(y_pred == class_num)[0]
#         X_class = X[indices]
#         y_class = y[indices]
#         print(X_class.shape, y_class.shape)
    
#         x_filename = f"/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/DT/WUSTL_Input1/x_test_adv_{attack_name}_Def{class_num}.npy"
#         y_filename = f"/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/DT/WUSTL_Input1/y_test_adv_{attack_name}_Def{class_num}.npy"
        
#         np.save(x_filename, X_class)
#         np.save(y_filename, y_class)
    
#         print(f"Save {class_num} to {attack_name}_Def{class_num}.npy")

#     end_time = time.time()
#     result = end_time - start_time
#     print(f"Execution Time: {result:.6f} seconds")
    

In [11]:
import numpy as np


attack_names = [
    "baseline", 
    "BIM", "FGSM", "PGD", "DF",
    "AutoPGD", "ZOO", "CaFA", "SINIFGSM", "VNIFGSM"
]

all_attack_list = []

model_name_list = ["XGB", "RF", "DT"]

for m_name in model_name_list:
    for attack_name in attack_names:
        each = []
        num = 0
        each.append(f"{m_name}1")
        for i in range(1, 12):
            y_filename = f"/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/{m_name}/WUSTL_Input1/x_test_adv_{attack_name}_Def{i}.npy"
            try:
                y = np.load(y_filename)
                num+=y.shape[0]
                each.append(y.shape[0])
            except FileNotFoundError:
                each.append(0)
        each.append(num)
        all_attack_list.append(each)


In [12]:
import pandas as pd
all_attack_list = pd.DataFrame(all_attack_list)
all_attack_list.to_csv("./Dynamite_Num_Attack_Input1.csv")

In [13]:
all_attack_list

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,XGB1,0,0,7,0,0,237193,0,1553,0,0,0,238753
1,XGB1,64451,0,252596,0,1439,170181,0,224811,398,2383,0,716259
2,XGB1,444825,0,13431,0,11776,12876,0,225535,2,7814,0,716259
3,XGB1,64451,0,252596,0,1439,170181,0,224811,398,2383,0,716259
4,XGB1,6,0,6541,10,11964,687436,0,3392,0,6910,0,716259
5,XGB1,84545,0,555054,0,171,54885,0,21551,7,46,0,716259
6,XGB1,0,0,28,0,0,711568,0,4663,0,0,0,716259
7,XGB1,3436,0,1323,32129,297,59234,0,619301,512,27,0,716259
8,XGB1,435725,0,59004,0,380,1146,0,220003,0,1,0,716259
9,XGB1,436151,0,40008,0,2107,13737,0,223329,0,927,0,716259
